In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
# Import the presplit data
X_train = pd.read_csv('data/X_train.csv', index_col='id')
X_test = pd.read_csv('data/X_test.csv', index_col='id')
y_train = pd.read_csv('data/y_train.csv', index_col='id')
y_test = pd.read_csv('data/y_test.csv', index_col='id')

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40095 entries, 45522 to 9914
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             40095 non-null  float64
 1   funder                 40095 non-null  object 
 2   gps_height             40095 non-null  int64  
 3   installer              40095 non-null  object 
 4   longitude              40095 non-null  float64
 5   latitude               40095 non-null  float64
 6   num_private            40095 non-null  int64  
 7   basin                  40095 non-null  object 
 8   region                 40095 non-null  object 
 9   region_code            40095 non-null  int64  
 10  district_code          40095 non-null  int64  
 11  population             40095 non-null  int64  
 12  public_meeting         40095 non-null  object 
 13  scheme_management      40095 non-null  object 
 14  permit                 40095 non-null  object 
 15 

In [4]:
# X_train = pd.get_dummies(X_train, drop_first=True)

# X_test = pd.get_dummies(X_test, drop_first=True)

In [5]:
y_train = np.array(y_train)
y_train = y_train.reshape(40095, )

In [6]:
y_test = np.array(y_test)
y_test = y_test.reshape(13365, )

In [7]:
rfc = RandomForestClassifier()
ss = StandardScaler()

X_train_nums = X_train.select_dtypes(exclude=object)
X_test_nums = X_test.select_dtypes(exclude=object)

X_train_nums_scaled = ss.fit_transform(X_train_nums)
X_test_nums_scaled = ss.transform(X_test_nums)

In [8]:
rfc.fit(X_train_nums_scaled, y_train)

preds = rfc.predict(X_test_nums_scaled)

In [9]:
train_scores = rfc.score(X_train_nums_scaled, y_train)
test_scores = rfc.score(X_test_nums_scaled, y_test)
print('The Mode\'s accuracy on the training data is', round(train_scores, 4))
print('The Model\'s accuracy on the test data is', round(test_scores, 4))

The Mode's accuracy on the training data is 0.9858
The Model's accuracy on the test data is 0.7149


In [10]:
log_loss_rfc = cross_val_score(rfc, X_train_nums_scaled, y_train, scoring='neg_log_loss', n_jobs = -1)
log_loss_rfc = -log_loss_rfc.mean()
print('Log Loss:', log_loss_rfc)

Log Loss: 1.0048775599566135


This inital model that only contains numerical data is severly overfitting the data. The next model will need to tune some hyperparameters to reduce the gap between the training and testing data. Our log loss has been decreased from the dummy classifer, but there is still room for improvement

In [11]:
pipline_1 = Pipeline([('ss', StandardScaler()), ('rfc', RandomForestClassifier())])

In [12]:
pipline_1.get_params()

{'memory': None,
 'steps': [('ss', StandardScaler()), ('rfc', RandomForestClassifier())],
 'verbose': False,
 'ss': StandardScaler(),
 'rfc': RandomForestClassifier(),
 'ss__copy': True,
 'ss__with_mean': True,
 'ss__with_std': True,
 'rfc__bootstrap': True,
 'rfc__ccp_alpha': 0.0,
 'rfc__class_weight': None,
 'rfc__criterion': 'gini',
 'rfc__max_depth': None,
 'rfc__max_features': 'auto',
 'rfc__max_leaf_nodes': None,
 'rfc__max_samples': None,
 'rfc__min_impurity_decrease': 0.0,
 'rfc__min_impurity_split': None,
 'rfc__min_samples_leaf': 1,
 'rfc__min_samples_split': 2,
 'rfc__min_weight_fraction_leaf': 0.0,
 'rfc__n_estimators': 100,
 'rfc__n_jobs': None,
 'rfc__oob_score': False,
 'rfc__random_state': None,
 'rfc__verbose': 0,
 'rfc__warm_start': False}

In [13]:
params_1 = {
    'rfc__max_depth': list(range(5,50,5)),
    'rfc__min_samples_leaf': list(range(5,50,5)),
    'rfc__n_estimators': list(range(25,500,25)),
}

In [14]:
rs_1 = RandomizedSearchCV(pipline_1, params_1, n_jobs=-1, verbose=2, n_iter=10)

rs_1.fit(X_train_nums, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  3.9min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('ss', StandardScaler()),
                                             ('rfc',
                                              RandomForestClassifier())]),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'rfc__max_depth': [5, 10, 15, 20, 25,
                                                           30, 35, 40, 45],
                                        'rfc__min_samples_leaf': [5, 10, 15, 20,
                                                                  25, 30, 35,
                                                                  40, 45],
                                        'rfc__n_estimators': [25, 50, 75, 100,
                                                              125, 150, 175,
                                                              200, 225, 250,
                                                              275, 300, 325,
                                                              

In [44]:
print('The train score for the random search with some hyperparameter tuning is',
      round(rs_1.score(X_train_nums, y_train), 4))
print('')
print('The test score for the random search with some hyperparameter tuning is',
      round(rs_1.score(X_test_nums, y_test), 4))

The train score for the random search with some hyperparameter tuning is 0.8255

The test score for the random search with some hyperparameter tuning is 0.7217


In [16]:
rs_1.best_params_

{'rfc__n_estimators': 325, 'rfc__min_samples_leaf': 5, 'rfc__max_depth': 25}

In [ ]:
log_loss_rfc = cross_val_score(rs_1, X_train_nums_scaled, y_train, scoring='neg_log_loss', n_jobs = -1)
log_loss_rfc = -log_loss_rfc.mean()
print('Log Loss:', log_loss_rfc)

# Model 1

This is he first model that will contain the categorical colums, there will be no hyperparameter tuning done yet. 

In [18]:
X_train_nums = X_train.select_dtypes(exclude = object)

X_train_cat = X_train.select_dtypes('object')

In [31]:
numerical_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(handle_unknown = 'ignore'))
])


In [32]:
ohe = OneHotEncoder(handle_unknown='ignore')

ohe.fit(X_train)

OneHotEncoder(handle_unknown='ignore')

In [33]:
trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cat.columns)
], verbose=True, n_jobs=-1)

In [34]:
model_pipe = Pipeline(steps=[
    ('trans', trans),
    ('rfr', RandomForestClassifier(verbose=1, n_jobs=-1))
])

In [35]:
model_pipe.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   33.3s finished


Pipeline(steps=[('trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))...
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantit

In [36]:
model_pipe.score(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


0.9972565157750343

In [37]:
model_pipe.score(X_test, y_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


0.7954358398802843

In [48]:
cv_1 = cross_val_score(model_pipe, X_train, y_train, scoring='neg_log_loss', n_jobs = -1, verbose = 1)
-cv_1.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


0.8541020610545565

In [47]:
cv_1 = cross_val_score(model_pipe, X_test, y_test, scoring='neg_log_loss', n_jobs = -1, verbose = 1)
-cv_1.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   21.7s remaining:   32.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   21.8s finished


0.8861059237679315

This first model is a significant improvement from the first Random Forst Classifer model. The train score increase by $0.0736$. However, the model is still significanly overfitting and that will need to be reduced by hyperparameter tuning. 

The log loss also improved from $1.005$ with the first model with only numeric colums to $0.8861$ with all colums included. This was an improvement of $0.1188$.